In [ ]:
%matplotlib qt5
import numpy as np
from image_analysis_lastversion08_04_2025 import InteractiveImageAnalysis

font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

stacks = ['/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/stack.h5']
stacks_ssb = ['/home/somar/Desktop/2025/Data for publication/Sample 2525/SSB reconstruction of 4d STEM data/stack_ssbs.h5']
stacks_ssb1 = ['/home/somar/Desktop/2025/Data for publication/Sample 2525/SSB reconstruction of 4d STEM data/stack.h5']

stacks_adf = ['/home/somar/Desktop/2025/Data for publication/Sample 2525/ADF images/stack.h5']

stack = stacks[0]
s = '/home/somar/test /stack.h5'
image_analysis = InteractiveImageAnalysis(stacks_ssb1[0], metadata_path=None, analysing_features=False, save_images_with_calibrated_scalebar=True, fixed_length_scalebar=True,
                                           clean_graphene_analysis=True, contamination_analysis=False, clusters_analysis=True , defects_analysis=False, font_path=font_path)

In [ ]:
from imagesequence import ImageSequence
filepath = '/home/somar/Desktop/2025/Data for publication/Sample 2475/ADF images/SuperScan-MAADF-3_2025-02-26T153114.686333_1024x1024_5.ndata1'
fp_pkl = '/home/somar/Desktop/2025/Data for publication/Sample 2475/ADF images/stack.pkl'
img = ImageSequence(fp_pkl)
data = img.raw_data
metadata = img.raw_metadata
fov = img.get_specific_metadata('fov_nm')
fov

In [ ]:
%matplotlib qt5
import numpy as np
from image_analysis_lastversion08_04_2025 import InteractiveImageAnalysis

font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

stacks = ['/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/stack.h5',
          '/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2525/stack.h5' ]
stack_asem = '/home/somar/Desktop/2025/ASEM/images for asem/stack.h5'

stack = stacks[0]

image_analysis = InteractiveImageAnalysis(stack, metadata_path=None, analysing_features=False, save_images_with_calibrated_scalebar=True, fixed_length_scalebar=True,
                                           clean_graphene_analysis=True, contamination_analysis=False, clusters_analysis=True , defects_analysis=False, font_path=font_path)

In [ ]:
%matplotlib qt5
import numpy as np
from image_analysis_lastversion08_04_2025 import InteractiveImageAnalysis

font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 


stack_labcourse = ['/home/somar/lab course/stack.h5','/home/somar/lab course/VCR1.h5']

stack = stack_labcourse[1]

image_analysis = InteractiveImageAnalysis(stack, metadata_path=None, analysing_features=False, save_images_with_calibrated_scalebar=True, fixed_length_scalebar=True,
                                           clean_graphene_analysis=True, contamination_analysis=False, clusters_analysis=True , defects_analysis=False, font_path=font_path)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
df_path = 'cl1_1000-1100C.csv'
df = pd.read_csv(df_path)
df

In [ ]:
print(df['Roundness'].mean())
print(df['Roundness'].std())

In [ ]:
import re
import ast
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
df_path = 'cl2_1100-1200C.csv'
df = pd.read_csv(df_path)
df
print("=== Raw Column Sample Inspection ===")
for col in ['Circularities', 'Roundness', 'Feret_Diameter', 'Aspect_Ratio']:
    print(f"\n{col} sample values:")
    print(df[col].dropna().astype(str))

# ---------- Safe float extraction ----------
def safe_extract_float(x):
    try:
        # Handle stringified lists, arrays, or scalars
        if isinstance(x, str):
            # Extract all floats using regex2023!CYcle@
            
            floats = re.findall(r"[-+]?\d*\.\d+|\d+", x)
            return float(floats[0]) if floats else np.nan
        elif isinstance(x, list) and len(x) > 0:
            return float(x[0])
        elif np.isscalar(x):
            return float(x)
        else:
            return np.nan
    except Exception:
        return np.nan

# ---------- Apply cleaning ----------
df['Circularity'] = df['Circularities'].apply(safe_extract_float)
df['Roundness'] = df['Roundness'].apply(safe_extract_float)
df['Feret_Diameter'] = df['Feret_Diameter'].apply(safe_extract_float)
df['Aspect_Ratio'] = df['Aspect_Ratio'].apply(safe_extract_float)

# ---------- Physical constants ----------
pixel_size = 0.063325  # nm/pixel
pixel_area_nm2 = pixel_size ** 2
pixel_length_nm = pixel_size

# ---------- Error estimation ----------
df['Area_error'] = pixel_area_nm2 * np.sqrt(df['Total_Cluster_Area_nm2'] / pixel_area_nm2)

# Estimate perimeter for error propagation
df['Perimeter'] = np.sqrt(4 * np.pi * df['Total_Cluster_Area_nm2'] / df['Circularity'].replace(0, np.nan))

# Partial derivatives for Circularity = 4πA / P²
P = df['Perimeter'].replace([np.inf, -np.inf], np.nan)
A = df['Total_Cluster_Area_nm2']

dC_dA = 4 * np.pi / (P ** 2)
dC_dP = -8 * np.pi * A / (P ** 3)

sigma_A = df['Area_error']
sigma_P = pixel_length_nm * np.sqrt(P / pixel_length_nm)

df['Circularity_error'] = np.sqrt((dC_dA * sigma_A) ** 2 + (dC_dP * sigma_P) ** 2)

# Roundness = 4A / (π D²)
D = df['Feret_Diameter'].replace([0, np.inf, -np.inf], np.nan)
dR_dA = 4 / (np.pi * D ** 2)
dR_dD = -8 * A / (np.pi * D ** 3)

sigma_D = pixel_length_nm  # 1 pixel uncertainty
df['Roundness_error'] = np.sqrt((dR_dA * sigma_A) ** 2 + (dR_dD * sigma_D) ** 2)

# Feret Diameter error
df['Feret_Diameter_error'] = sigma_D

# Aspect Ratio error (assume 5%)
df['Aspect_Ratio_error'] = 0.05 * df['Aspect_Ratio']

# ---------- Diagnostics ----------
print("\n=== Error Term Summaries ===")
for col in ['Circularity_error', 'Roundness_error', 'Feret_Diameter_error', 'Aspect_Ratio_error']:
    print(f"{col}: min={df[col].min():.4f}, max={df[col].max():.4f}, nan={df[col].isna().sum()}")

print("\n=== Metric Value Summaries ===")
for col in ['Circularity', 'Roundness', 'Feret_Diameter', 'Aspect_Ratio']:
    print(f"{col}: min={df[col].min():.4f}, max={df[col].max():.4f}, nan={df[col].isna().sum()}")

# ---------- Plot: Area vs Slice ----------
plt.figure(figsize=(10, 4))
# plt.errorbar(df['Slice'], df['Total_Cluster_Area_nm2'],
#              yerr=df['Area_error'], fmt='o-', color='navy',
#              ecolor='gray', elinewidth=1.5, capsize=4,
#              markerfacecolor='white', markeredgewidth=1.5, markersize=6)
plt.plot(df['Slice'], df['Total_Cluster_Area_nm2'],
         'o-', color='navy', markerfacecolor='white',
         markeredgewidth=1.5, markersize=6)


plt.title('Cluster Area changes with heating', fontsize=12)
plt.xlabel('Frame Number', fontsize=11)
plt.ylabel('Area (nm²)', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# ---------- Plot: Shape Metrics ----------
fig, axs = plt.subplots(4, 1, figsize=(10, 12), sharex=True)

metrics = ['Circularity', 'Roundness', 'Feret_Diameter', 'Aspect_Ratio']
errors = ['Circularity_error', 'Roundness_error', 'Feret_Diameter_error', 'Aspect_Ratio_error']
colors = ['darkgreen', 'darkorange', 'firebrick', 'purple']

for ax, metric, error, color in zip(axs, metrics, errors, colors):
    y = df[metric].replace([np.inf, -np.inf], np.nan)
    yerr = df[error].replace([np.inf, -np.inf], np.nan)

    if y.dropna().empty:
        ax.set_title(f'{metric} – No valid data', fontsize=10)
        ax.set_ylim(0, 1)
        continue

    # ax.errorbar(df['Slice'], y, yerr=yerr,
    #             fmt='o-', color=color, ecolor='lightgray',
    #             elinewidth=1.4, capsize=3,
    #             markerfacecolor='white', markeredgewidth=1.5, markersize=6)
    ax.plot(df['Slice'], y,
        'o-', color=color, markerfacecolor='white',
        markeredgewidth=1.5, markersize=6)


    valid_y = y.dropna()
    data_range = valid_y.max() - valid_y.min()
    if data_range == 0:
        ax.set_ylim(valid_y.iloc[0] - 0.1, valid_y.iloc[0] + 0.1)
    else:
        ax.set_ylim(valid_y.min() - 0.1 * data_range,
                    valid_y.max() + 0.1 * data_range)

    ax.set_ylabel(metric.replace('_', ' '), fontsize=10)
    ax.grid(alpha=0.2)

axs[-1].set_xlabel('Frame Number', fontsize=11)
plt.suptitle('Shape Metrics changes with heating', fontsize=13)
plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.show()


In [ ]:
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Safe parsing of shape descriptors from stringified lists or values
def parse_descriptor(x):
    if pd.isnull(x):
        return np.nan
    if isinstance(x, list):
        if len(x) > 0:
            try:
                return float(x[0])
            except:
                return np.nan
        else:
            return np.nan
    if isinstance(x, (int, float)):
        return float(x)
    if isinstance(x, str):
        try:
            val = ast.literal_eval(x)
            if isinstance(val, (list, tuple)):
                if len(val) > 0:
                    try:
                        return float(val[0])
                    except:
                        return np.nan
                else:
                    return np.nan
            elif isinstance(val, (int, float)):
                return float(val)
        except (ValueError, SyntaxError):
            pass
        try:
            return float(x)
        except:
            return np.nan
    return np.nan

# Apply parsing to shape descriptor columns
shape_cols = ['Circularities', 'Roundness', 'Feret_Diameter', 'Aspect_Ratio']
for col in shape_cols:
    if col in df.columns:
        df[col] = df[col].apply(parse_descriptor)

# Compute errors
pixel_size = 0.063325  # nm per pixel
if 'Total_Cluster_Area_nm2' in df.columns:
    df['Area_error'] = np.sqrt(df['Total_Cluster_Area_nm2'].abs()) * pixel_size**2

for col in shape_cols:
    if col in df.columns:
        df[col + '_err'] = 0.02 * df[col].abs()

# Plot Total Cluster Area vs Slice with error bars
fig1, ax1 = plt.subplots(figsize=(6, 4))
color_area = 'tab:blue'
x = df['Slice']
if 'Total_Cluster_Area_nm2' in df.columns and 'Area_error' in df.columns:
    y = df['Total_Cluster_Area_nm2']
    yerr = df['Area_error']
    ax1.errorbar(x, y, yerr=yerr, fmt='o-', color=color_area, ecolor='gray', capsize=3)
ax1.set_xlabel('Slice')
ax1.set_ylabel('Total Cluster Area (nm^2)')
ax1.set_title('Cluster Area vs. Slice')
# Adjust y-axis limits with some margin
if 'Total_Cluster_Area_nm2' in df.columns:
    y_vals = df['Total_Cluster_Area_nm2']
    y_err = df.get('Area_error', np.zeros_like(y_vals))
    mask = ~np.isnan(y_vals)
    if mask.any():
        y_min = np.nanmin(y_vals[mask] - y_err[mask])
        y_max = np.nanmax(y_vals[mask] + y_err[mask])
        margin = (y_max - y_min) * 0.1
        ax1.set_ylim(max(0, y_min - margin), y_max + margin)
plt.tight_layout()
plt.show()

# Plot shape metrics vs Slice in a 2x2 grid with error bars
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.ravel()
shape_titles = ['Circularity', 'Roundness', 'Feret Diameter (nm)', 'Aspect Ratio']

for ax, col, title in zip(axes, shape_cols, shape_titles):
    x = df['Slice']
    if col in df.columns and col + '_err' in df.columns:
        y = df[col]
        yerr = df[col + '_err']
        ax.errorbar(x, y, yerr=yerr, fmt='o-', color='tab:orange', ecolor='gray', capsize=3)
    ax.set_title(title)
    ax.set_xlabel('Slice')
    ax.set_ylabel(title)
    # Adjust y-axis limits for each subplot
    if col in df.columns:
        y = df[col]
        yerr = df.get(col + '_err', np.zeros_like(y))
        mask = ~np.isnan(y)
        if mask.any():
            y_min = np.nanmin(y[mask] - yerr[mask])
            y_max = np.nanmax(y[mask] + yerr[mask])
            margin = (y_max - y_min) * 0.1 if y_max != y_min else y_max * 0.1
            ax.set_ylim(y_min - margin, y_max + margin)
plt.tight_layout()
plt.show()
